In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import seaborn as sns
import json

In [2]:
exp_folders = [
    'exp_regression_kfold_1',
    'exp_regression_kfold_2',
    'exp_regression_kfold_3',
    'exp_regression_kfold_4'
]

In [3]:
DATA_001 = '0.01'
DATA_005 = '0.05'
JSON_DATA = [DATA_001, DATA_005, "0.1", "0.15", "0.2"]
WIDTH = 0.4
WIDTH_F = 0.01

all_avg_data = {}


for single_exp_folder in exp_folders:
    folders = glob(os.path.join(single_exp_folder, '*_exp'))

    acc_data = None

    all_data_avg_f = []
    for single_folder in folders:
        epoch_folders = glob(os.path.join(single_folder, 'acc_*'))
        # Find max in all folders
        for single_epoch_folder in epoch_folders:
            # Check acc
            with open(os.path.join(single_epoch_folder, 'acc_data.json'), 'r') as fp: 
                new_data = json.load(fp)

            if acc_data is None:
                acc_data = new_data
                continue

            if new_data[DATA_001] > acc_data[DATA_001]:
                acc_data = new_data
        all_data_avg_f.append(acc_data)

    # N maximum values
    # Calcualte avg
    folder_name = single_folder.split('/')[0]
    all_avg_data[folder_name] = {}
    for json_name in JSON_DATA:
        single_json_data = [single_elem[json_name] for single_elem in all_data_avg_f]
        all_avg_data[folder_name][json_name] = sum(single_json_data) / len(single_json_data)

fig, ax = plt.subplots()   
for i, (single_f, val_f) in enumerate(all_avg_data.items()):
    x = list(map(lambda x: float(x) + (WIDTH/len(all_avg_data))*(i - len(all_avg_data)//2), list(range(1, len(val_f)+1))))
    y = list(val_f.values())
    ax.bar(x, y, width=WIDTH, label=f'Цикл-обучения номер {i+1}')#label=str(single_f))
ax.legend(loc='best')
ax.set_facecolor('seashell')
ax.set_xticks(list(range(1, len(val_f) + 1)))
ax.set_xticklabels(JSON_DATA)
ax.set_xlabel('Параметр доверительного интервала', fontsize=16)
ax.set_ylabel('Точность', fontsize=16)
fig.set_facecolor('floralwhite')
fig.set_figwidth(12)    #  width Figure
fig.set_figheight(12)    #  height Figure
fig.savefig(f'avg_data_folder.png', facecolor=fig.get_facecolor(), edgecolor='none')
# Clear plt figures
plt.close('all')

# Avg at all folders
fig, ax = plt.subplots()
x = list(map(lambda x: float(x), JSON_DATA))
y = np.mean(np.asarray([
    [val_f[single] for single in JSON_DATA]
    for val_f in all_avg_data.values()
]), axis=0)

ax.bar(x, y, width=WIDTH_F)
ax.set_xticks(x)
ax.set_xticklabels(JSON_DATA)
ax.set_facecolor('seashell')
ax.set_xlabel('Параметр доверительного интервала', fontsize=14)
ax.set_ylabel('Точность', fontsize=14)
for height_y, width_x in zip(y, x):
    ax.text(width_x-WIDTH_F, height_y+WIDTH_F, str(round(height_y, 2)), color='black', fontweight='bold')
fig.set_facecolor('floralwhite')
fig.set_figwidth(8)    #  width Figure
fig.set_figheight(8)    #  height Figure
fig.savefig(f'avg_data_folder_{single_f}.png', facecolor=fig.get_facecolor(), edgecolor='none')
# Clear plt figures
plt.close('all')